In [25]:
# Give yourself access to common
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# Setup autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
# Gather training and testing data
from common import *
from sklearn.model_selection import train_test_split

nba = get_cleaned_external_data()
# nba = get_cleaned_baseline_data()
nba = add_log_y_values(nba)
X_train, X_test, y_train, y_test = split_data(nba)

In [27]:
# Fit the model
from sklearn.neural_network import MLPRegressor
import numpy as np

# MLPClassifier only classifies data as integers or strings, therefore, our problem is one of regression for the neural network
# Consequently I should use the mlp regressor
mlp = MLPRegressor(hidden_layer_sizes=(30,30,10,10), solver='adam', max_iter=1000)
mlp.fit(X_train,y_train)


MLPRegressor(hidden_layer_sizes=(30, 30, 10, 10), max_iter=1000)

In [28]:
# Tune the Hyerparameters
from sklearn.model_selection import GridSearchCV
# Define the hyperparameters you want to tune
params = {
    'hidden_layer_sizes': [(10,), (20,), (30,), (40,), (50,), (10,10,10), (20,20,20)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01, 0.1]
}

# Use GridSearchCV to search over the hyperparameter grid
grid_search = GridSearchCV(mlp, param_grid=params, cv=5, n_jobs=-1)

# Fit the GridSearchCV object on the training data
grid_search.fit(X_train, y_train)

# Print the best set of hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model on the validation set using the best hyperparameters
mlp = grid_search.best_estimator_

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
import numpy as np
from sklearn.metrics import mean_squared_error
predict_test = mlp.predict(X_test)
test_set_rsquared = mlp.score(X_test, y_test)
test_set_rmse = np.sqrt(mean_squared_error(predict_test, y_test))
print('R_squared value: ', test_set_rsquared) # Variances are not nicely correlated
print('RMSE: ', test_set_rmse) # Fairly good at data prediction


R_squared value:  -999.5445595745944
RMSE:  39.47740017372302


In [ ]:
# Save Model
import pickle
from common import NN_FILENAME
pickle.dump(mlp, open('saved_models/' + NN_FILENAME, 'wb'))